In [69]:
import pandas as pd
import csv
import json

In [77]:
fileName = "Cuphead.json"

In [78]:
def defineMap(fileName):
    with open('../Data/KeyMapping/'+fileName, 'r') as f:
        keyMapping = json.load(f)
        return keyMapping

In [79]:
try:
    keyMapping = defineMap(fileName)
except:
    print("json file does not exist")

In [80]:
def readFile(dataFile):
    df = pd.read_csv("../Data/UserInput/"+dataFile, sep = '\t', header=None)
    actionCol = df.iloc[:,-1]
    return actionCol

In [81]:
def clean_inputs(keyCol):
    keyCol = keyCol.str.replace("Button\.", '', regex=True)
    keyCol = keyCol.str.replace("Key\.", '', regex=True)
    return keyCol

In [82]:
def annotateData(dataFile):                                              #open the file to be annotated
    df = pd.read_csv("../Data/UserInput/"+dataFile, sep = '\t', header=None)                             #create a dataframe of file that needs annotation
    keyCol = df.iloc[:,-2]                                                          #this only takes the column of user inputs (ex. w, a, s, d)
    cleanedKey = clean_inputs(keyCol)
    keyCol = cleanedKey.copy()
    keyCol = keyCol.map(keyMapping).to_frame().fillna("other")                     #match the user inputs with the key_mapping json data, then convert it to a dataframe and replace all NaN values with other
    df2 = pd.merge(df, keyCol, left_index=True, right_index=True)                   #merge the original file that needs annotations with the annotations
    df2.insert(1, "col1",cleanedKey)
    df2 = df2.rename({'1_x': 'keys', '1_y': 'action'}, axis=1)
    df2 = df2.drop("keys", axis = 1)
    df2 = df2.drop(df2[df2.action =="other"].index)
    display(df2)
    df2.to_csv("../Data/AnnotatedInput/annotated"+dataFile,sep = '\t', header=None, index = False)

In [83]:
try:
    annotateData("cupheadkb.tsv")
except:
    print("Failed to annotate keyboard data")
    


,0,col1,2,action
0,2023-04-13 12:27:08.979268,z,pressed,jump
1,2023-04-13 12:27:09.075052,z,released,jump
2,2023-04-13 12:27:11.361745,z,pressed,jump
3,2023-04-13 12:27:11.526767,z,released,jump
4,2023-04-13 12:27:12.411511,z,pressed,jump
...,...,...,...,...
1130,2023-04-13 12:31:01.834850,right,released,right
1131,2023-04-13 12:31:01.857937,left,pressed,left
1132,2023-04-13 12:31:02.111777,left,released,left
1133,2023-04-13 12:31:02.438257,x,released,shoot
